In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import warnings
warnings.filterwarnings(action='ignore')
import sys

import matplotlib.pylab as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
plt.rc('font',family=font_name)
mpl.rcParams['axes.unicode_minus']=False

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score

### Data 불러오기

In [2]:
data = pd.read_csv('data_preprocessing.csv')
data

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,1,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,1,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,1,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,1,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,1,60,19,20,1433500
...,...,...,...,...,...,...,...,...,...,...,...
2691680,2020,3,충북,충주시,휴양콘도 운영업,1,1,30,3,4,43300
2691681,2020,3,충북,충주시,휴양콘도 운영업,1,1,40,3,3,35000
2691682,2020,3,충북,충주시,휴양콘도 운영업,1,1,50,4,6,188000
2691683,2020,3,충북,충주시,휴양콘도 운영업,1,1,60,3,3,194000


### Label Encoding

In [3]:
dtypes = data.dtypes
data_col = data.loc[:,['CARD_SIDO_NM','STD_CLSS_NM','CARD_CCG_NM']]
encoders = {}

for col in data_col:
    if str(dtypes[col]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data[col])
        encoders[col] = encoder
        
df_num = data.copy()        
for col in encoders.keys():
    encoder = encoders[col]
    df_num[col] = encoder.transform(data[col])

df_num=df_num.drop(['CSTMR_CNT','CNT'],axis=1)
df_num

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,AMT
0,2019,1,0,3,0,1,1,20,311200
1,2019,1,0,3,0,1,1,30,1374500
2,2019,1,0,3,0,1,1,40,2764300
3,2019,1,0,3,0,1,1,50,3690830
4,2019,1,0,3,0,1,1,60,1433500
...,...,...,...,...,...,...,...,...,...
2691680,2020,3,16,202,40,1,1,30,43300
2691681,2020,3,16,202,40,1,1,40,35000
2691682,2020,3,16,202,40,1,1,50,188000
2691683,2020,3,16,202,40,1,1,60,194000


### Train, Test 구분

In [4]:
Train_data = df_num.loc[(df_num['year']==2020)&((df_num['month']==1)|(df_num['month']==2)|
                                              (df_num['month']==3))]
Train_data_X = Train_data.drop(['AMT'],axis=1)
print('Train_data :',Train_data.shape)

Train_data : (521750, 9)


### SET Pred_Dataframe

In [7]:
temp         = []

CARD_SIDO_NM = Train_data_X['CARD_SIDO_NM'].unique()
STD_CLSS_NM  = Train_data_X['STD_CLSS_NM'].unique()
CARD_CCG_NM  = Train_data_X['CARD_CCG_NM'].unique()
SEX          = Train_data_X['CARD_CCG_NM'].unique()
Region_diff  = Train_data_X['region_diff'].unique()
AGE          = Train_data_X['AGE'].unique()
year         = [2020]
month        = [4, 7]
a=[]
for sido in CARD_SIDO_NM:
    for std in STD_CLSS_NM:
        for ccg in CARD_CCG_NM:
            for age in AGE:
                
                for sex in SEX:
                    for region in Region_diff:
                        for i in year:
                            for j in month:
                temp.append([ sido, ccg,std, age])

temp = np.array(temp)

In [ ]:
temp         = []

CARD_SIDO_NM = Train_data_X['CARD_SIDO_NM'].unique()
STD_CLSS_NM  = Train_data_X['STD_CLSS_NM'].unique()
# CARD_CCG_NM  = Train_data_X['CARD_CCG_NM'].unique()
SEX          = Train_data_X['CARD_CCG_NM'].unique()
Region_diff  = Train_data_X['region_diff'].unique()
AGE          = Train_data_X['AGE'].unique()
year         = [2020]
month         = [4, 7]

for sido in CARD_SIDO_NM:
    CARD_CCG_NM  = Train_data_X['CARD_CCG_NM'].unique()
    for std in STD_CLSS_NM:
        for ccg in CARD_CCG_NM:
            for age in AGE:
                for sex in SEX:
                    for region in Region_diff:
                        for i in year:
                            for j in month:
                                temp.append([ i, j, sido, ccg,std, sex, region, age])

temp = np.array(temp)

In [ ]:
adf= []
for tem in temp:
    for sex in SEX:
        for region in Region_diff:
            for i in year:
                for j in month:
                    adf.append([i,j,tem,sex,region])
adf = np.array(adf)

In [11]:
temp

array([[  0,   3,   0,  30],
       [  0,   3,   0,  40],
       [  0,   3,   0,  50],
       ...,
       [ 16, 202,  30,  70],
       [ 16, 202,  30,  20],
       [ 16, 202,  30,  10]], dtype=int64)

In [8]:
pred_data = pd.DataFrame(data= temp, columns= Train_data_X.columns)
pred_data

ValueError: Shape of passed values is (1107533, 4), indices imply (1107533, 8)

In [6]:
# CCG = np.array(Train_data_en.filter(regex = 'CARD_CCG_NM_').columns)
# SEX= np.array(Train_data_en.filter(regex = 'SEX_').columns)
# REGION = np.array(Train_data_en.filter(regex = 'region_diff_').columns)
# AGE = np.array(Train_data_en.filter(regex = 'AGE_').columns)

# pred_col = np.hstack(['year','month','CARD_SIDO_NM','STD_CLSS_NM',CCG,SEX,REGION,AGE])

# pred_data = pd.DataFrame(columns = pred_col)
# pred_data

,year,month,CARD_SIDO_NM,STD_CLSS_NM,CARD_CCG_NM_가평군,CARD_CCG_NM_강남구,CARD_CCG_NM_강동구,CARD_CCG_NM_강릉시,CARD_CCG_NM_강북구,CARD_CCG_NM_강서구,...,region_diff_1,region_diff_2,region_diff_3,AGE_10,AGE_20,AGE_30,AGE_40,AGE_50,AGE_60,AGE_70


In [7]:
test_enc = LabelEncoder()
test_enc.fit(CCG)
CCG_enc = test_enc.transform(CCG)

test_enc = LabelEncoder()
test_enc.fit(AGE)
AGE_enc = test_enc.transform(AGE)

test_enc = LabelEncoder()
test_enc.fit(SEX)
SEX_enc = test_enc.transform(SEX)

test_enc = LabelEncoder()
test_enc.fit(REGION)
REGION_enc = test_enc.transform(REGION)

In [11]:
temp = []
CARD_SIDO_NMs = Train_data['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = Train_data['STD_CLSS_NM'].unique()
year          = [2020]
month         = [4, 7]

for sido in CARD_SIDO_NMs:
    for std in STD_CLSS_NMs:
        for ccg in CCG_enc:
            for age in AGE_enc:
                for sex in SEX_enc:
                    for region in REGION_enc:
                        for i in year:
                            for j in month:
                                temp.append([ i, j, sido, std,ccg, sex, region, age])
                                
pred_data_1 = pd.DataFrame(data= temp, columns= pred_data.columns)

ValueError: 243 columns passed, passed data had 8 columns

### Encoding

In [5]:
dum_col = ['CARD_CCG_NM','SEX','region_diff','AGE']

Train_data_en = pd.get_dummies(Train_data, columns=dum_col)
# Train_data_en = Train_data_en.drop(['year'], axis = 1)

display(Train_data_en)

,year,month,CARD_SIDO_NM,STD_CLSS_NM,AMT,CARD_CCG_NM_가평군,CARD_CCG_NM_강남구,CARD_CCG_NM_강동구,CARD_CCG_NM_강릉시,CARD_CCG_NM_강북구,...,region_diff_1,region_diff_2,region_diff_3,AGE_10,AGE_20,AGE_30,AGE_40,AGE_50,AGE_60,AGE_70
2169935,2020,1,0,0,1903450,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
2169936,2020,1,0,0,1845900,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
2169937,2020,1,0,0,6132260,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
2169938,2020,1,0,0,3162700,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2169939,2020,1,0,0,554000,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2691680,2020,3,16,40,43300,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2691681,2020,3,16,40,35000,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2691682,2020,3,16,40,188000,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2691683,2020,3,16,40,194000,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


### Model

In [ ]:
### 세팅중

sido_list = Train_data_en['CARD_SIDO_NM'].unique()
std_list = Train_data_en['STD_CLSS_NM'].unique()
temp = []

for i, sido in enumerate(sido_list):
    for j, std in enumerate(std_list): 
        #if((sido==0)&(std==4)):
        print("★ ", i, j)
        df_train1 = Train_data_en[(Train_data['CARD_SIDO_NM'] == sido) & (Train_data_en['STD_CLSS_NM'] == std)]
#         df_test1 = Test_data_en[(Test_data['CARD_SIDO_NM'] == sido) & (Test_data_en['STD_CLSS_NM'] == std)]

        #csv파일 저장
        #df_train1.to_csv('C:\\Users\\acorn\\Desktop\\train.csv',encoding = 'utf-8-sig', index = False)
        #df_test1.to_csv('C:\\Users\\acorn\\Desktop\\test.csv',encoding = 'utf-8-sig', index = False)

        #DF가 비어있을때 학습X
        if( df_train1.empty): 
            print("train가 비어있음")
            continue

        y_train = df_train1['AMT'].to_numpy(dtype=np.float32).reshape(-1,1)
        X_train = df_train1.drop(['AMT'],axis=1).to_numpy(dtype=np.float32)
#         y_test = df_test1['AMT'].to_numpy(dtype=np.float32).reshape(-1,1)
#         X_test = df_test1.to_numpy(dtype=np.float32)        

        print("sido: {}번째 / std: {}번째".format(sido, std))
        print('TRAIN',X_train.shape,y_train.shape)
#         print('TEST',X_test.shape,y_test.shape)
        print('-'*30)

        #num_hidden_units = X_test.shape[1]
        num_hidden_units = 200
        input_dim = X_train.shape[1]

        model = Sequential()
        model.add(Dense(num_hidden_units,input_dim = input_dim, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units, kernel_initializer='normal',activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
#         model.add(Dropout(0.25))
        model.add(Dense(1))
        model.compile(loss='MeanAbsolutePercentageError', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=1)
        
        # 예측
        pred = model.predict(pred_data)
        pred = np.expm1(pred)
        temp['AMT'] = np.round(pred, 0)
        temp['REG_YYMM'] = temp['year']*100 + temp['month']
        temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
        temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
        
#         # 디코딩 
#         temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
#         temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])
        
        # 제출 파일 만들기
        submission = pd.read_csv('submission_mingsu.csv', index_col=0)
        submission = submission.drop(['AMT'], axis=1)
        submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
        submission.index.name = 'id'
        submission.to_csv('submission_mingsu.csv', encoding='utf-8-sig')
        submission.head()
        
        
#         y_pred = model.predict(X_test).flatten()
#         print('예측값 : ', y_pred[0])
#         print('실제값 : ', y_test[0])

#         scores = model.evaluate(X_test, y_test, verbose=1)
#         print("Baseline Error: %.2f%%" % (100-scores[1]*100))
        print("-"*60)

### 리스트 확인

In [ ]:
# print(sido_list)
# print(std_list)
# display(df_num[df_num['STD_CLSS_NM']==12])
# display(df_num[df_num['STD_CLSS_NM']==4])
# df_num[df_num['STD_CLSS_NM']==4].to_csv('C:\\Users\\acorn\\Desktop\\test.csv',encoding = 'utf-8-sig', index = False)

### 기타작업

In [ ]:
Train_data[(Train_data['CARD_SIDO_NM'] == 0) & (Train_data['STD_CLSS_NM'] == 4)]

### 저장

In [ ]:
data_final1.to_csv('data_preprocessing_2.csv',encoding = 'utf-8-sig', index = False)

In [ ]:
data_final1 = pd.read_csv('data_preprocessing_2.csv')
data_final1.head()